In [1]:
# required installs:
# numpy, pandas, seaborn, selenium, chromedriver-py, requests, beautifulsoup4

# all installs can be installed with pip 
#  pip install [package]

# if chromedriver-py doesn't work, dl from here (https://sites.google.com/a/chromium.org/chromedriver/home)

import numpy as np
import pandas as pd
# import seaborn as sns

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from chromedriver_py import binary_path # this will get you the path variable

import requests
from bs4 import BeautifulSoup

import time
# import lxml
# from lxml import etree

In [ ]:
# use just beautifulsoup to open webpage
# didn't work too well, not gonna use it
url = 'https://www.amazon.com/gp/css/order-history?ref_=nav_AccountFlyout_orders'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, 'lxml')
print(soup.prettify())

In [ ]:
# use selenium to open webpage


driver = webdriver.Chrome(executable_path=binary_path)
driver.get("https://www.amazon.com/gp/css/order-history?ref_=nav_AccountFlyout_orders")
try:
    #driver.manage().timeouts().implicitlyWait(5, TimeUnit.SECONDS);

    # !!! CLEAR THESE BEFORE PUSHING !!!
    email = driver.find_element_by_name("email")
    email.clear()
    email.send_keys("") # my info left out for safety
    email.send_keys(Keys.RETURN)

    pwd = driver.find_element_by_name("password")
    pwd.clear()
    pwd.send_keys("") # my info left out for safety
    pwd.send_keys(Keys.RETURN)
    
    #num_pages = driver.find_element_by_class_name('')
    #first_order = driver.find_element_by_class_name('a-box-group a-spacing-base order')
    orders_box = driver.find_element_by_id('ordersContainer')
    
    # first try (leaving this here just cuz)
#     orders = orders_box.find_elements_by_class_name('a-box-group')
#     #for i in orders:   
    
#     #finding prices
#     prices = []
#     for i in orders_box.find_elements_by_class_name('a-color-price'):
#         #print(i.text)
#         prices.append(i.text)
#     print('\n')
    
#     # finding product names
#     order_names = []
#     for i in orders_box.find_elements_by_class_name('a-link-normal'):
#         if((i.text != '') & (i.text != 'Order Details') & (i.text != 'Invoice') & (i.text != 'Archive order')):
#             #print(i.text)
#             order_names.append(i.text)
    
    
    
    
    
    # second try
    orders = orders_box.find_elements_by_class_name('shipment')
    prices = []
    order_names = []
    order_qtys = []
    for i in orders:
        shipment_prices = i.find_elements_by_class_name('a-color-price')
        shipment_order_names = i.find_elements_by_class_name('a-link-normal')
        #while '' in shipment_order_names:
        shipment_order_names = [j for j in shipment_order_names if j.text != ''] 
        #print(shipment_order_names)
        shipment_order_qtys = i.find_elements_by_class_name('item-view-qty')
        for j in range(len(shipment_prices)):
            prices.append(shipment_prices[j].text)
            order_names.append(shipment_order_names[j].text)
            if(j < len(shipment_order_qtys)):
                order_qtys.append(shipment_order_qtys[j].text)
            else:
                order_qtys.append('1')
    
    
    time.sleep(10)
    
finally:
    driver.quit()
print(prices)
#print(order_names)  # my info left out for safety
#print(order_qtys) 
driver.quit()

['$5.99', '$15.99', '$19.39', '$9.67', '$19.49', '$39.99', '$4.25', '$20.89', '$7.49', '$9.29', '$6.99', '$6.99']


In [46]:
len(prices)

12

In [ ]:
info_d = {'Item': order_names, 'Price': prices, 'Quantity':order_qtys}
df = pd.DataFrame(info_d)
df
# my output left out for safety

In [61]:
df.to_csv('orders.csv')
# my csv not pushed for safety